In [1]:
from torch.utils.tensorboard import SummaryWriter
import json
import time
%matplotlib inline
from deep_rl_ga.ga_env import GeneticAlgorithmEnv
from deep_rl_ga.agent import Agent
from deep_rl_ga.memory import Experience, ReplayMemory, extract_tensors
from deep_rl_ga.strategy import EpsilonGreedyStrategy
from deep_rl_ga.network import DQN
from deep_rl_ga.qvalues import QValues

from deap import benchmarks
from deap import tools

import torch
import torch.optim as optim
import torch.nn.functional as F

from itertools import count
import random

import numpy as np

import matplotlib.pyplot as plt
from IPython import display

from deap import creator
from typing import List
import itertools
# import cuml
# import cudf
# import cupy
import datetime
import os
from deep_rl_ga.diversity import (
    Clusterer,
    fitness_max_mean_ratio_diversity,
    fitness_mean_min_ratio_diversity,
    gene_mean_std_diversity,
    gene_mean_unique_ratio_diversity,
    number_of_clusters_diversity,
    selBestPossible,
    selWorstPossible,
)

%env OMP_NUM_THREADS 14

# Genetic algorithm params
IND_SIZE = 3
LOW_BOUND = -5.12
UP_BOUND = 5.12
FITNESS_FUNCTION = benchmarks.sphere
SEARCH_SPACE_CENTER = np.array([(UP_BOUND - LOW_BOUND / 2) for _ in range(IND_SIZE)])
SEARCH_SPACE_DIAMETER = np.linalg.norm(np.array([UP_BOUND for _ in range(IND_SIZE)]) - np.array([LOW_BOUND for _ in range(IND_SIZE)]))

INITIAL_POPULATION_SIZE = 150

# Crossover + Mutation params
ATTRIBUTE_MUTATION_RATE = 0.3

# Selection params
TOURNAMENT_SIZE = 3
TOP_BEST_SIZE = 25

TOURNAMENT_SIZE_VALUES = [3, 10]
TOP_BEST_SIZE_VALUES = [25]

NUM_TRAP_ACTIONS = 1

MAX_EVALS = 10_000

_NUM_GENERATIONS = MAX_EVALS / INITIAL_POPULATION_SIZE

RANDOM_SEED = 127

random.seed(
    RANDOM_SEED
    )
np.random.seed(
    RANDOM_SEED
    )

ACTIONS_SEL = list(itertools.chain(
    [{'function': selWorstPossible} for _ in range(NUM_TRAP_ACTIONS)],  # fill the population with the WORST individuals - this is the action the network should avoid
    [{'function': tools.selTournament, 'tournsize': tsize} for tsize in TOURNAMENT_SIZE_VALUES],
    # [{'function': tools.selRoulette}],
    # [{'function': tools.selStochasticUniversalSampling}],
    # [{'function': tools.selAutomaticEpsilonLexicase}],
    # [{'function': lambda population, k: tools.selWorst(population, TOP_BEST_SIZE)}],
    # [{'function': selBestPossible, 'top_n': TOP_BEST_SIZE}],  # fill the population with the BEST individuals
    # [{'function': lambda population, k: population}],  # no-op
))

ACTIONS_CX = [
    {'function': tools.cxBlend, 'alpha': UP_BOUND},
    # {'function': tools.cxUniform, 'indpb': 0.4},
    # {'function': tools.cxOrdered},
    # {'function': lambda ind1, ind2: (ind1, ind2)},  # no-op
]

ACTIONS_MU = [
    {'function': tools.mutGaussian, 'mu': 0, 'sigma': UP_BOUND / 2, 'indpb': ATTRIBUTE_MUTATION_RATE},
    # {'function': tools.mutPolynomialBounded, 'eta': 0.3, 'low': LOW_BOUND, 'up': UP_BOUND, 'indpb': ATTRIBUTE_MUTATION_RATE},
    # {'function': tools.mutShuffleIndexes, 'indpb': ATTRIBUTE_MUTATION_RATE},
    # {'function': lambda ind: (ind,)},  # no-op
]

ACTIONS_CXPB = [0.1, 0.3, 0.6]

ACTIONS_MUTPB = [0.05, 0.2, 0.6]


CLUSTERER = Clusterer()

N_CLUSTERS = 10

N_STACKED_STATES = 5

# class RapidsClusteringMethod:
#     def __init__(self, n_clusters, random_state):
#         self.model = cuml.cluster.KMeans(init='k-means||', n_clusters=n_clusters, random_state=random_state)
#
#     def fit_predict(self, X):
#         device_data = cudf.DataFrame(X)
#         return self.model.fit_predict(device_data).to_numpy()

STAT_FUNCTIONS = [
    ("clusters_of_multiple_fns", CLUSTERER.clusters_of_fns([
        fitness_max_mean_ratio_diversity,
        fitness_mean_min_ratio_diversity,
        gene_mean_std_diversity,
        gene_mean_unique_ratio_diversity,
        lambda p: len(p) / INITIAL_POPULATION_SIZE,  # Cluster population size as part of initial population size
        lambda p: np.linalg.norm(np.mean(p, axis=0) - SEARCH_SPACE_CENTER) / SEARCH_SPACE_DIAMETER,  # Cluster centroid distance from the middle of search space; normalized by search space diameter
    ], n_clusters=N_CLUSTERS, random_seed=RANDOM_SEED)),
]

writer = SummaryWriter()

# Neural network params
batch_size = 256
gamma = 0.85
target_update = 5
memory_size = 100_000
lr = 3e-4
num_episodes = 750

eps_start = 1
eps_end = 0.0001
eps_decay = (eps_start - eps_end) / (num_episodes * _NUM_GENERATIONS)

curr_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
em = GeneticAlgorithmEnv(
    num_dims=IND_SIZE,
    low_bound=LOW_BOUND,
    up_bound=UP_BOUND,
    fitness_fn=benchmarks.rastrigin,
    max_evals=MAX_EVALS,
    initial_population_size=INITIAL_POPULATION_SIZE,
    actions_sel=ACTIONS_SEL,
    actions_cx=ACTIONS_CX,
    actions_mu=ACTIONS_MU,
    actions_cxpb=ACTIONS_CXPB,
    actions_mutpb=ACTIONS_MUTPB,
    stat_functions=STAT_FUNCTIONS,
    clusterer=CLUSTERER,
    device=curr_device,
    number_of_stacked_states=N_STACKED_STATES,
    optimum_fitness=0.0,
    optimum_fitness_delta=0.01,
)

# RL setup
strategy = EpsilonGreedyStrategy(eps_start, eps_end, eps_decay)
agent = Agent(strategy, em.num_actions_available(), curr_device)
memory = ReplayMemory(memory_size)

policy_net = DQN(em.get_num_state_features(), em.num_actions_available()).to(curr_device)
target_net = DQN(em.get_num_state_features(), em.num_actions_available()).to(curr_device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
optimizer = optim.Adam(params=policy_net.parameters(), lr=lr)


# Saving model data
save_every_num_episodes = 150
curr_datetime = datetime.datetime.utcnow()
save_dir_path = f'../{curr_datetime.day}-{curr_datetime.month}-{curr_datetime.year}_{curr_datetime.hour}-{curr_datetime.minute}_{em.number_of_stacked_states}-states_{em.num_actions_available()}-actions_seed-{RANDOM_SEED}_with-{NUM_TRAP_ACTIONS}-trap-actions'
os.mkdir(save_dir_path)

# Save simulation parameters
print('Saving environment parameters...')
with open(os.path.join(save_dir_path, 'available_actions.txt'), 'w') as f:
    for idx, (sel_idx, cx_idx, mu_idx, cxpb_idx, mutpb_idx) in enumerate(em.actions):
        f.write(f'[{idx}] ({em.actions_sel[sel_idx]}, {em.actions_cx[cx_idx]}, {em.actions_mu[mu_idx]}, {em.actions_cxpb[cxpb_idx]}, {em.actions_mutpb[mutpb_idx]})\n')

with open(os.path.join(save_dir_path, 'env_params_dump.json'), 'w') as f:
    f.write(em.to_json())

with open(os.path.join(save_dir_path, 'agent_params_dump.json'), 'w') as f:
    f.write(agent.to_json())
print('Environment parameters have been saved')

episode_best_fitnesses = []
for episode in range(num_episodes):
    start_episode = time.perf_counter()
    em.reset()
    state = em.get_state()

    take_action_time = 0
    memory_sampling_time = 0
    inference_time = 0
    network_updating_time = 0
    q_values_time = 0
    for timestep in count():
        action = agent.select_action(state, policy_net)
        start = time.perf_counter()
        reward = em.take_action(action)  # this is a performance bottleneck, it takes 0.1seconds, almost 100% of the time required for a single timestep
        stop = time.perf_counter()
        take_action_time += (stop - start)
        next_state = em.get_state()
        memory.push(Experience(state, action, next_state, reward))

        writer.add_scalar('Reward', reward, episode * (em.max_evals // em.initial_population_size) + timestep)
        writer.add_scalar('Best fitness', em.curr_best_fitness, episode * (em.max_evals // em.initial_population_size) + timestep)
        writer.add_scalar('Fitness sum', em.curr_fitness_sum, episode * (em.max_evals // em.initial_population_size) + timestep)
        writer.add_scalar('Action', action, episode * (em.max_evals // em.initial_population_size) + timestep)


        state = next_state

        if memory.can_provide_sample(batch_size):
            start = time.perf_counter()
            experiences = memory.sample(batch_size)
            states, actions, rewards, next_states = extract_tensors(experiences, curr_device)
            stop = time.perf_counter()
            memory_sampling_time += (stop - start)

            start = time.perf_counter()
            current_q_values = QValues.get_current(policy_net, states, actions)
            stop = time.perf_counter()
            inference_time += (stop - start)

            start = time.perf_counter()
            next_q_values = QValues.get_next(target_net, next_states, curr_device)
            target_q_values = (next_q_values * gamma) + rewards
            stop = time.perf_counter()
            q_values_time += (stop - start)

            start = time.perf_counter()
            loss = F.mse_loss(current_q_values, target_q_values.unsqueeze(1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            stop = time.perf_counter()
            network_updating_time += (stop - start)

        if em.done:
            # TODO: This should be a value that we want to track across episodes, e.x. number of generations before global optimum was found
            episode_best_fitnesses.append(em.hof[0].fitness.values[0])
            writer.add_scalar('Episode Best Fitness', em.hof[0].fitness.values[0], episode)
            writer.add_scalar('Episode Total No. of evaluations', em.max_evals - em.evals_left, episode)
            print('Episode', len(episode_best_fitnesses), '\n')
            # plot(episode_best_fitnesses, 100)
            break
    stop_episode = time.perf_counter()
    print(f'\n\nEpisode took {stop_episode - start_episode}\n\n')
    print(f'Taking action took {take_action_time} seconds in total')
    print(f'Sampling memory took {memory_sampling_time} seconds in total')
    print(f'Inference took {memory_sampling_time} seconds in total')
    print(f'Getting target Q-Values took {q_values_time} seconds in total')
    print(f'Updating network took {network_updating_time} seconds in total')

    if episode % target_update == 0:
        target_net.load_state_dict(policy_net.state_dict())

    # Checkpoint model data
    if episode > 0 and episode % save_every_num_episodes == 0:
        # Parameters
        torch.save(policy_net, os.path.join(save_dir_path, f'policy_net_{episode}_episodes'))
        torch.save(target_net, os.path.join(save_dir_path, f'target_net_{episode}_episodes'))

        # Performance
        moving_averages = [np.mean(episode_best_fitnesses[i-100:i]) if i > 99 else np.mean(episode_best_fitnesses[:i]) for i, _ in enumerate(episode_best_fitnesses)]
        plt.plot(moving_averages)
        plt.savefig(os.path.join(save_dir_path, f'moving_averages_{episode}_episodes.png'))
writer.close()
    # TODO: This should be a value that we want to track across episodes, e.x. number of generations before global optimum was found
    # if get_moving_average(100, episode_best_fitnesses)[-1] <= 0.0001:
    #     break

env: OMP_NUM_THREADS=14
Saving environment parameters...
Environment parameters have been saved


/home/pbilko/miniconda3/envs/PP/lib/python3.9/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646756402876/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0
/home/pbilko/miniconda3/envs/PP/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/pbilko/miniconda3/envs/PP/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or

Episode 1 



Episode took 1.0942471259986633


Taking action took 1.065433332012617 seconds in total
Sampling memory took 0 seconds in total
Inference took 0 seconds in total
Getting target Q-Values took 0 seconds in total
Updating network took 0 seconds in total


/home/pbilko/miniconda3/envs/PP/lib/python3.9/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/pbilko/miniconda3/envs/PP/lib/python3.9/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/pbilko/miniconda3/envs/PP/lib/python3.9/site-packages/numpy/core/_methods.py:256: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/pbilko/miniconda3/envs/PP/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/pbilko/miniconda3/envs/PP/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/pbilko/miniconda3/envs/PP/lib

Episode 2 



Episode took 0.9027486300037708


Taking action took 0.8770108970056754 seconds in total
Sampling memory took 0 seconds in total
Inference took 0 seconds in total
Getting target Q-Values took 0 seconds in total
Updating network took 0 seconds in total


...

/home/pbilko/miniconda3/envs/PP/lib/python3.9/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/pbilko/miniconda3/envs/PP/lib/python3.9/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/pbilko/miniconda3/envs/PP/lib/python3.9/site-packages/numpy/core/_methods.py:256: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/pbilko/miniconda3/envs/PP/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/pbilko/miniconda3/envs/PP/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/pbilko/miniconda3/envs/PP/lib

Episode 30 



Episode took 1.9894211190039641


Taking action took 0.8647844189690659 seconds in total
Sampling memory took 0.06202317700081039 seconds in total
Inference took 0.06202317700081039 seconds in total
Getting target Q-Values took 0.22124247802275931 seconds in total
Updating network took 0.5937181550179957 seconds in total


/home/pbilko/miniconda3/envs/PP/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/pbilko/miniconda3/envs/PP/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/pbilko/miniconda3/envs/PP/lib/python3.9/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/pbilko/miniconda3/envs/PP/lib/python3.9/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmea

Episode 31 



Episode took 2.117648134000774


Taking action took 0.8944816490620724 seconds in total
Sampling memory took 0.0663310319869197 seconds in total
Inference took 0.0663310319869197 seconds in total
Getting target Q-Values took 0.2269537279571523 seconds in total
Updating network took 0.6680856150123873 seconds in total


/home/pbilko/miniconda3/envs/PP/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/pbilko/miniconda3/envs/PP/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/pbilko/miniconda3/envs/PP/lib/python3.9/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/pbilko/miniconda3/envs/PP/lib/python3.9/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmea